In [ ]:
"""
Алгоритм программы
1) Считываются файлы тренда и всех атрибутов. По ним схранются значения по координатам, размерность сетки и сами координаты
2) Проверяется правильно ли сделана разметка сеток и выставлены координаты всех файлов. Если где-то не сходятся координтаы, то необьходимо отправить пользователя переделать карты
3) Т.к. карта содержит очень большое количество точек, то будем брать значения с некоторой периодичностью (шагом)

"""

In [111]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

def multiple_regression_for_lateral_trend(file_lateral_trend, folder_pass_of_atributs, XX = 0.95):
    """
    file_lateral_trend - пкть к файлу lateral_trend
    folder_pass_of_atributs - путь к папке со всеми атрибутами
    XX - значение R2, которое пользователь может сам задать (по дефолту 0,95)
    """
    def open_coord(file_lateral_trend):
        with open(file_lateral_trend, 'r') as f:
            coord = []
            for line in f:
                if line.startswith('# Grid_space'):
                    Grid_space = line[13:-1].split(',')
                if line.startswith('# Grid_size'):
                    Grid_size = line[12:-1].split(' x ')
                if not line.startswith('#'):  # Пропуск строк, начинающихся с #
                    x, y, z, i, j = line.split()  # Разбиение строки по столбцам
                    x = float(x)
                    y = float(y)
                    z = float(z) 
                    i = int (i)
                    j = int(j)
                    coord.append([x, y, z, i, j])  # Добавление данных в словарь

        return coord, Grid_space, Grid_size
    
    def atribusts(folder_pass_of_atributs):
        """
        Проходимся по всем атрибутам в папке и собираем словарь в котором хранятся все данные атрибутов.
        Словарь{название файла : {data : [], Grid_space : [], Grid_size :[]}, ...}
        """
        Surf = {}
        files = os.listdir(folder_pass_of_atributs)
    
        # Проходим по каждому файлу и создаем словари
        for file in files:
            if file.endswith('.txt'):
                var_name = file[:-4]  # Убираем '.txt'
        
                with open(os.path.join(folder_pass_of_atributs, file), 'r') as f:
                    coord = []
                    for line in f:
                        if line.startswith('# Grid_space'):
                            Grid_space = line[13:-1].split(',')
                        if line.startswith('# Grid_size'):
                            Grid_size = line[12:-1].split(' x ')
                        if not line.startswith('#'):  # Пропуск строк, начинающихся с #
                            x, y, z, i, j = line.split()  # Разбиение строки по столбцам
                            x = float(x)
                            y = float(y)
                            z = float(z) 
                            i = int (i)
                            j = int(j)
                            coord.append([x, y, z, i, j])  # Добавление данных в словарь

                #   Создаем словарь для каждого файла с тремя ключами
                Surf[var_name] = {
                    'data': coord,
                    'Grid_space': Grid_space,
                    'Grid_size': Grid_size
                }
        return Surf
    
    def proverka_kart(karta_space, karta_size, atribut_space, atribut_size):
        if karta_space == atribut_space:
            if karta_size == atribut_size:
                print('координаты карты и размерности сетки совпадают')
            else:
                print('размерности сетки не совпадают, переделайте сетку')
        else:
            print('координаты карты не совпадают, переделайте карту')
    
    def proseivanie(list, h):
        """
        Т.к. карта содержит очень большое количество точек, то данная функция берет значения с некоторой периодичностью (шагом)
        h - шаг перехода
        """
        result = []
        start = list[0][3]
        i = list[0][3]
        j = list[0][4]
        for k in range(len(list)):
            if list[k][3] == start and list[k][4] == start:
                result.append(list[k][2])
                i += h
            elif list[k][3] != i and list[k][4] == j:
                continue
            elif list[k][3] == i and list[k][4] == j:
                result.append(list[k][2])
                i += h
            elif list[k][4] != j + h:
                continue
            elif list[k][3] == start and list[k][4] ==j + h:
                result.append(list[k][2])
                i = start + h
                j += h
        return result

        #Удаляем выбросы по 1 и 3 квартилю (все что ниже 25% и выше 75%, теоретически можно потом изменять процентили)
    def remove_outliers(x, y):
        q1_x = np.percentile(x, 25)
        q3_x = np.percentile(x, 75)
        iqr_x = q3_x - q1_x
    
        q1_y = np.percentile(y, 25)
        q3_y = np.percentile(y, 75)
        iqr_y = q3_y - q1_y
    
        x_filtered = []
        y_filtered = []
    
        for xi, yi in zip(x, y):
            if (xi >= (q1_x - 1.5 * iqr_x)) and (xi <= (q3_x + 1.5 * iqr_x)) and \
            (yi >= (q1_y - 1.5 * iqr_y)) and (yi <= (q3_y + 1.5 * iqr_y)):
                x_filtered.append(xi)
                y_filtered.append(yi)
    
        return np.array(x_filtered), np.array(y_filtered)
        
    
        
    def zavisimoste (list_1, list_2, name_1, name_2, flag = 0):
        """"
        list_1, list_2 - списки, которые проверяем
        name_1, name_2 - имена этих списков
        flag показывает нужно ли выводить график 0 если нет, 1 если да
        """
               

        #списки  без выбросов
        list_1_n, list_2_n = remove_outliers(list_1, list_2) 
        
        # Преобразуем списки в numpy массивы
        X = np.array(list_1_n).reshape(-1, 1)  # Признаки (независимая переменная)
        y = np.array(list_2_n)  # Целевая переменная

        # Линейная регрессия
        linear_model = LinearRegression()
        linear_model.fit(X, y)
        #y_pred_linear = linear_model.predict(X)
        r_squared_linear = round(linear_model.score(X, y), 5)
        results = {
            'equation': f"y = {linear_model.intercept_:.2f} + {linear_model.coef_[0]:.2f} * x",
            'r_squared': r_squared_linear
        }
        y_pred_linear_2 = [linear_model.intercept_ + linear_model.coef_[0]*x for x in list_1_n]
        
        
        if flag == 1:
            print ("Уравнение регрессии:", results['equation'])
            print ("значение R^2:", results['r_squared'])
        
            fig = go.Figure()
            fig.update_layout(title=f'Сравнение {name_1} и {name_2}')
        
            fig.add_trace(go.Scatter(
                x=list_1_n,
                y=list_2_n,
                mode='markers',
                name='Данные',
                marker=dict(color='blue')
            ))
    
            # Линейная регрессия
            fig.add_trace(go.Scatter(
                x=list_1_n,
                y=y_pred_linear_2,
                mode='lines',
                name='Линейная регрессия',
                line=dict(color='red')
            ))
            
            # Показываем график
            fig.show()
        
        
        return results
    
    def multiple_regression_equation(Y, X):
        """
        Функция для вычисления уравнения множественной регрессии.
        
        :param Y: Список значений зависимой переменной.
        :param X: Словарь, где ключи - имена независимых переменных, а значения - списки значений.
        :return: Уравнение множественной регрессии в виде строки.
        """
        
        # Создаем DataFrame для независимых переменных
        X_df = pd.DataFrame(X)
        
        # Добавляем константу (свободный член) в модель
        X_df = sm.add_constant(X_df)
        
        # Преобразуем зависимую переменную в массив
        Y = np.array(Y)
        
        # Создаем модель
        model = sm.OLS(Y, X_df)
        
        # Фитируем модель
        results = model.fit()
        
        # Получаем коэффициенты
        coefficients = results.params
        
        # Формируем строку уравнения       
        equation = f"Y = {coefficients.iloc[0]:.4f}"  # Свободный член
        for i in range(1, len(coefficients)):
            variable_name = X_df.columns[i]
            equation += f" + {coefficients.iloc[i]:.4f} * {variable_name}"
        
        return equation  

    Lateral_trend, Lateral_trend_Grid_space, Lateral_trend_Grid_size = open_coord(file_lateral_trend)
    Surface_attributes = atribusts(folder_pass_of_atributs)
    for key in Surface_attributes:
        print(f'Для Lateral_trend и для {key}')
        proverka_kart(Lateral_trend_Grid_space, Lateral_trend_Grid_size, Surface_attributes[key]['Grid_space'], Surface_attributes[key]['Grid_size'])
    
    Lateral_trend_z = proseivanie(Lateral_trend, 2)
    Surface_attributes_z = {}
    for key in Surface_attributes:
        Surface_attributes_z[key] = proseivanie(Surface_attributes[key]['data'], 2)
    
    all_data_files = {**{"Lateral_trend": Lateral_trend_z}, **Surface_attributes_z}
    Table_R2 = {}
    names = list(all_data_files.keys())
    for key in names:
        Table_R2[key] = {}
    
    for i in range(len(names)):
    
        for j in range(i, len(names)):
            if names[i] == names[j]:
                continue
            else:
                R2 = zavisimoste(all_data_files[names[i]], all_data_files[names[j]], names[i], names[j], 0)['r_squared']
                Table_R2[names[i]][names[j]] = R2
                Table_R2[names[j]][names[i]] = R2
    
    #Вывод таблицы
    df = pd.DataFrame(Table_R2)
    df = df.T
    df.fillna(0, inplace=True)
    print(df)
    
   
    parametrs_in_model = []
    for key in Table_R2['Lateral_trend']:
        if Table_R2['Lateral_trend'][key] >=XX:
            parametrs_in_model.append(key)
    
    zavisimie_atributs = []
    del_atributs = []
    if len(parametrs_in_model) > 1:
        for i in range(len(parametrs_in_model)-1):
            for j in range(i+1, len(parametrs_in_model)):           
                if Table_R2[parametrs_in_model[i]][parametrs_in_model[j]] >= XX:
                    zavisimie_atributs.append([parametrs_in_model[i], parametrs_in_model[j]])
                    
    if len(zavisimie_atributs) > 0:
        for i in range(len(zavisimie_atributs)):
            print(f" Атрибуты {zavisimie_atributs[i][0]} и {zavisimie_atributs[i][1]} взаимокоррелируют. Их нельзя использовать совместно для построения карты тренда (ошибка прогноза искусственно завышена). Исключите наименее представительный атрибут")
            print('Выберите какой из атрибутов убрать из модели регрессии и введите его название')
            del_atributs.append(input())
            
    name_atribut_in_model = [item for item in parametrs_in_model if item not in del_atributs]
    parametrs_in_model_final = {}
    for i in range(len(name_atribut_in_model)):
        parametrs_in_model_final[name_atribut_in_model[i]] = Surface_attributes_z[name_atribut_in_model[i]]
          
    equation = multiple_regression_equation(Lateral_trend_z, parametrs_in_model_final)

    return equation
    
    
    
    

In [110]:
equation = multiple_regression_for_lateral_trend('C:/HV/Seismic/datas/Lateral_trend_No_smooth_Heff.txt', 'C:/HV/Seismic/datas/atributs', 0.95)
print()
print(equation)

Для Lateral_trend и для 1_05_2_Top_Sweet_RMSh
координаты карты и размерности сетки совпадают
Для Lateral_trend и для 2_01_7_Fm_Amp_MaxM
координаты карты и размерности сетки совпадают
Для Lateral_trend и для 2_02_7_Fm_RMS_MaxM
координаты карты и размерности сетки совпадают
Для Lateral_trend и для 2_11_7_Fm_Amp_Contr_MaxM
координаты карты и размерности сетки совпадают
Для Lateral_trend и для AmplitudeContrast_HalfEnergy_TWT
координаты карты и размерности сетки совпадают
                                  1_05_2_Top_Sweet_RMSh  2_01_7_Fm_Amp_MaxM  \
Lateral_trend                                   0.08252             0.00374   
1_05_2_Top_Sweet_RMSh                           0.00000             0.20571   
2_01_7_Fm_Amp_MaxM                              0.20571             0.00000   
2_02_7_Fm_RMS_MaxM                              0.24584             0.93081   
2_11_7_Fm_Amp_Contr_MaxM                        0.05805             0.08512   
AmplitudeContrast_HalfEnergy_TWT                0.082